[View in Colaboratory](https://colab.research.google.com/github/e-longo/Fraudes-MLP/blob/master/Fraudes_MLP.ipynb)

# Identificação de fraudes em movimentações financeiras utilizando redes neurais

Trabalho de conclusão de curso da pós graduação em ciência de dados pelo IGTI

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
import matplotlib as mpl
import matplotlib.pyplot as plt
import keras as kr
from google.colab import files
import io

Using TensorFlow backend.


Importação da base de dados utilizada no trabalho através de uma biblioteca do google colab e em seguidas os dados são persistindo em um data frame do pandas

In [2]:
uploaded = files.upload()
df = pd.read_csv(io.StringIO(uploaded['bs140513_032310_full.csv'].decode('utf-8')))

#exibe os cinco primeiros registros
df.head()

Saving bs140513_032310_full.csv to bs140513_032310_full.csv


,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


É utilizada a função LabelEncoder da biblioteca preprocessing do sklearn para transformar os dados categóricos providos da base em dados numéricos. Para tipo de dado categórico é criado um número que serve como identificador único. Por exemplo: dentro do domínio de gênero (gender) existe Masculino e Feminino (M e F respectivamente) que são transformados em 1 e 2.

Há essa necessidade de transformação pois na hora de definir as camadas da rede neural a biblioteca trabalha apenas com valores numéricos.

In [3]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
data = df.apply(le.fit_transform)
data.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,210,4,2,0,30,0,12,455,0
1,0,2753,2,2,0,30,0,12,3968,0
2,0,2285,4,1,0,18,0,12,2689,0
3,0,1650,3,2,0,30,0,12,1725,0
4,0,3585,5,2,0,30,0,12,3572,0


No passo a seguir é dividida a base em dados de treinamento e dados de teste, sendo que cada um terá o tamanho de 80% e 20%, respectivamente, da quantidade total de registros.

In [0]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size = 0.2)

A partir da segmentação anterior de dados de treinamento e teste, são divididos mais uma vez, sendo que as variáveis X possuirão as features utilizadas no modelo e as variáveis Y conterão os resultados que queremos encontrar, ou seja, se é fraude ou não.

In [5]:
x_train = train.iloc[:,1:9].values[:]
y_train = train['fraud'].values[:]

x_test = test.iloc[:,1:9].values[:]
y_test = test['fraud'].values[:]

print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)

(475714, 8) (475714,)
(118929, 8) (118929,)


Etapa onde são definidas as camadas da rede neural, sendo que neste caso existe uma camada de entrada, três hidden layers e uma camada de saída.

In [6]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(8,)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='tanh'))
model.add(Dense(2, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               4608      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 22        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 3         
Total para

Compilação do modelo utilizando o otimizador rmsprop.

In [0]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Treinamento do modelo utilizando 10 epochs e 20% dos dados para uma nova etapa, a de validação

In [8]:
from keras.callbacks import ModelCheckpoint

#train the model
checkpointer = ModelCheckpoint(filepath='mnist.model.best.hdf5',
                               verbose=1,save_best_only=True)

hist = model.fit(x_train, y_train, batch_size=128, epochs=10,
                validation_split=0.2, callbacks=[checkpointer],
                verbose=1, shuffle=True)

Train on 380571 samples, validate on 95143 samples
Epoch 1/10
380571/380571 [==============================] - 18s 46us/step - loss: 0.2042 - acc: 0.9466 - val_loss: 0.0677 - val_acc: 0.9874

Epoch 00001: val_loss improved from inf to 0.06765, saving model to mnist.model.best.hdf5
Epoch 2/10
174976/380571 [============>.................] - ETA: 8s - loss: 0.0645 - acc: 0.9881

380571/380571 [==============================] - 18s 46us/step - loss: 0.0652 - acc: 0.9880 - val_loss: 0.0677 - val_acc: 0.9874

Epoch 00002: val_loss did not improve
Epoch 3/10
380571/380571 [==============================] - 18s 47us/step - loss: 0.0652 - acc: 0.9880 - val_loss: 0.0677 - val_acc: 0.9874

Epoch 00003: val_loss did not improve
Epoch 4/10
  1152/380571 [..............................] - ETA: 24s - loss: 0.0657 - acc: 0.9878 

380571/380571 [==============================] - 18s 47us/step - loss: 0.0652 - acc: 0.9880 - val_loss: 0.0677 - val_acc: 0.9874

Epoch 00004: val_loss did not improve
Epoch 5/10
325120/380571 [========================>.....] - ETA: 2s - loss: 0.0646 - acc: 0.9881

380571/380571 [==============================] - 18s 47us/step - loss: 0.0652 - acc: 0.9880 - val_loss: 0.0677 - val_acc: 0.9874

Epoch 00005: val_loss did not improve
Epoch 6/10
380571/380571 [==============================] - 17s 46us/step - loss: 0.0652 - acc: 0.9880 - val_loss: 0.0677 - val_acc: 0.9874

Epoch 00006: val_loss did not improve
Epoch 7/10
 37504/380571 [=>............................] - ETA: 14s - loss: 0.0606 - acc: 0.9890

380571/380571 [==============================] - 17s 46us/step - loss: 0.0652 - acc: 0.9880 - val_loss: 0.0677 - val_acc: 0.9874

Epoch 00007: val_loss did not improve
Epoch 8/10
356864/380571 [===========================>..] - ETA: 0s - loss: 0.0654 - acc: 0.9879

380571/380571 [==============================] - 17s 45us/step - loss: 0.0652 - acc: 0.9880 - val_loss: 0.0677 - val_acc: 0.9874

Epoch 00008: val_loss did not improve
Epoch 9/10
380571/380571 [==============================] - 17s 45us/step - loss: 0.0652 - acc: 0.9880 - val_loss: 0.0677 - val_acc: 0.9874

Epoch 00009: val_loss did not improve
Epoch 10/10
 48000/380571 [==>...........................] - ETA: 14s - loss: 0.0608 - acc: 0.9889

380571/380571 [==============================] - 17s 45us/step - loss: 0.0652 - acc: 0.9880 - val_loss: 0.0676 - val_acc: 0.9874

Epoch 00010: val_loss improved from 0.06765 to 0.06765, saving model to mnist.model.best.hdf5


In [0]:
history = model.fit(x_train, y_train,
                    batch_size=500,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

# https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
# https://datascience.stackexchange.com/questions/17516/how-to-deal-with-string-labels-in-multi-class-classification-with-keras
# http://www.ritchieng.com/machinelearning-one-hot-encoding/

Cálculo da acurácia do modelo aplicanco aos dados de teste da base

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100 * score[1]

print('Acurácia do teste: %.2f%%' %accuracy)


Acurácia do teste: 98.81%
